In [1]:
import qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, execute
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.tools.visualization import plot_histogram
from qiskit import IBMQ
import numpy as np


In qunautm world, the state of a particle is a superposition (linear combination) of states.  The coefficients can be complex and expressed in terms of $e^{i2\pi\phi}$, where $\phi$ is called the phase. The square of the coefficient gives the probability of the particle collapsed to each state under measurement.  The sum of squared coefficient must equal to 1. Taking a two-level system as an example, the state of a particle is the superposition of $0$ and $1$.  
$|\Psi\rangle=\alpha |0\rangle +\beta |1\rangle$  
$|\alpha|^2+|\beta|^2=1$

In [3]:
def create_superposition(phase):
# phase =0: create 0+1 superposition
# phase =1: create 0-1 superposition
# phase =0~1: create 0+ exp(i*2*pi*phase)1 superposition
    q=QuantumRegister(1)
    c=ClassicalRegister(1)
    cir=QuantumCircuit(q,c)
    if phase==1:
        cir.x(q)
    elif not phase ==0:
        cir.u1(2*np.pi*phase,q)
    cir.h(q)
    cir.measure(q,c)
    return cir

In [4]:
cir=create_superposition(0.5)
cir.draw()

In [5]:
# simulate on local classical computer
job=execute(cir, backend=Aer.get_backend('qasm_simulator'), shots=1024)
result=job.result()
print(result.get_counts())

{'1': 514, '0': 510}


No matter what phase chosen, the resultant state always has the same probability of getting 0 and 1. 
Measurement of the phase is not trivial. I will come to the derivation and explanation in a later jupyter notebook. For the readers who would like to take a glance on how to do it, I will take the phase=0.25 as an example. Others can skip the rest of this notebook and come back to this part later.
We will use the binary form of the phase for the computation, $(0.25)_{10}=(0.01)_{2}$.  The number at the least significant bit is 1, while the most significant bit is 0.     
Step1: Create a two bit circuit. Apply H to the first bit. The first bit is the control bit. The second bit is the superposition state which phase is to be measured.  
Step2: Create the control-unitary gate which phase=0.25.   
Step3: Apply the control-unitary gate on the two bit system 4 times. (Note that 4 is the inverse of $(0.01)_2$.)   
Step4: Apply H gate on the first bit.   
Step5: Apply measurement. The most probable state of the first bit should be 1. (If the number at the least significant bit is 0, then the most probable state would be 0. )   
It is worth noting that the bit order used in qiskit is: the first created bit is on the right hand side and is also the least significant bit. In other words, the controlled bit in this example is written on the right hand side. 


In [46]:
def control_u(cir,phase,cbit,tbit):
    cir.u1(-2*np.pi*phase,tbit)
    cir.cx(cbit,tbit)
    cir.u1(2*np.pi*phase, tbit)
    cir.cx(cbit, tbit)
    

In [44]:
def measure_phase(phase):
    q=QuantumRegister(2)
    c=ClassicalRegister(2)
    cir=QuantumCircuit(q,c)
    cir.h(q[0])
    cir.u1(2*np.pi*phase,q[1])
    control_u(cir, phase, q[0], q[1])
    control_u(cir, phase,q[0],q[1])
    cir.h(q[0])
    cir.measure(q[0],c[0])
    return cir

In [56]:
cir=measure_phase(0.25)
job=execute(cir, backend=Aer.get_backend('qasm_simulator'), shots=1024)
result=job.result()
print(result.get_counts())

{'01': 1024}
